In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import json, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
import torch
import os
import cv2

cfg = get_cfg()
cfg.merge_from_file("model-eskperimen5/configmodel5.yml")
cfg.MODEL.WEIGHTS = "model-eskperimen5/content/output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55   # set the testing threshold for this model (0.6)
# cfg.DATASETS.TEST = ("my_dataset_test", )
#cfg.DATASETS.TEST = ("my_dataset_val", )

cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)

imageName = "./kemejadanchinosRGB.png"
im = cv2.imread(imageName)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], scale=0.8)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])
# print("success")

output_image = out.get_image()[:, :, ::-1]  # Convert RGB to BGR
cv2.imwrite("./output_prediction.png", output_image)
print("success, image saved")

[06/18 12:19:52 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from model-eskperimen5/content/output/model_final.pth ...


/home/duevano/miniconda3/envs/detectron2-env/lib/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


success, image saved


In [2]:
from transformers import Blip2Processor

# load the pretrain model?
checkpoint = 'Salesforce/blip2-opt-2.7b'
processor = Blip2Processor.from_pretrained(checkpoint)

/home/duevano/miniconda3/envs/detectron2-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
from transformers import Blip2ForConditionalGeneration
from peft import PeftModel, PeftConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BASE model hanya sekali
base_model = Blip2ForConditionalGeneration.from_pretrained(
    "ybelkada/blip2-opt-2.7b-fp16-sharded",  # sesuaikan dengan base dari semua peft_model_id kamu
    torch_dtype=torch.float16,
    cache_dir="./cache-blip2"
).to(device)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.06it/s]


In [4]:
# Fungsi untuk switch adapter
def load_adapter(base_model, peft_model_id, device):
    model = PeftModel.from_pretrained(base_model, peft_model_id)
    model = model.to(device)
    return model

# Contoh penggunaan:
# Ganti adapter dengan cepat
peft_model_id_1 = "content/data/blip2_visual_model_1v2_nonpadded"
peft_model_id_2 = "content/data/blip2_visual_model_1v2"

# Ganti adapter sesuai kebutuhan
# model = load_adapter(base_model, peft_model_id_1, device)

# ... lakukan inferensi
model = load_adapter(base_model, peft_model_id_2, device)
# ... lakukan inferensi lagi

In [5]:
# Set OpenAI API key
import os
from openai import OpenAI

# Set API key for OPENAI LLM
os.environ["OPENAI_API_KEY"] = '' # masukkan api key LLM, bisa pakai key dari LLM berbayar atau key gratis dari github models

client = OpenAI()

In [39]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# max length caption generated by blip-2
MAX_LENGTH = 16
# batch size for blip-2 model
CAPTION_BATCH_SIZE = 32  # number of crops per batch for BLIP-2 inference
LLM_MODEL = "gpt-4o-mini"

# Specify which segmentation classes to include (remember! index start from 0)
CHOSEN_CLASSES = [4, 5, 6, 7, 8, 9, 10]

# --- Helper Functions ---
def segment_image_in_memory(image: np.ndarray):
    """
    Perform instance segmentation and return list of crop arrays and metadata, without writing to disk.
    """
    outputs = predictor(image)
    instances = outputs["instances"].to("cpu")
    masks = instances.pred_masks.numpy()
    boxes = instances.pred_boxes.tensor.numpy().astype(int)
    classes = instances.pred_classes.numpy()
    scores = instances.scores.numpy()
    crops_meta = []
    for i, (mask, box, cls, score) in enumerate(zip(masks, boxes, classes, scores)):
        # if cls not in CHOSEN_CLASSES: continue # skip crops if filtered
        x1, y1, x2, y2 = box
        crop = image[y1:y2, x1:x2]
        mask_crop = (mask[y1:y2, x1:x2] * 255).astype(np.uint8)
        crop_masked = cv2.bitwise_and(crop, crop, mask=mask_crop)
        pil_img = Image.fromarray(cv2.cvtColor(crop_masked, cv2.COLOR_BGR2RGB))
        crops_meta.append({"image": pil_img, "class": int(cls), "score": float(score)})
    return crops_meta


def batch_caption_inference(crops_meta, processor, model, device, max_length, batch_size):
    """
    Run BLIP-2 captioning on crops in batches. Returns list of captions aligned to crops_meta.
    """
    captions = []
    for i in range(0, len(crops_meta), batch_size):
        batch = crops_meta[i:i+batch_size]
        images = [item["image"] for item in batch]
        inputs = processor(images=images, return_tensors="pt").to(device, torch.float16)
        out_ids = model.generate(pixel_values=inputs.pixel_values, max_length=max_length)
        batch_caps = processor.batch_decode(out_ids, skip_special_tokens=True)
        captions.extend(batch_caps)
    return captions


def combine_instance_captions(captions_list):
    # remove empty and duplicates
    unique = list(dict.fromkeys([c for c in captions_list if c and c.strip()]))
    return ", ".join(unique)

# LLM combination (optional)
def llm_combine(caption_str: str):
    # <A> no punctuation, no "person is wearing" => ideal for scoring
    # message_content = (
    #     "I have a list of captions that describe each segmented part of a fully clothed person in an image. "
    #     "The list of captions is: " + caption_str + ". "
    #     "Your task is to combine these captions into one cohesive caption that describes "
    #     "the clothing worn by the person in the image concisely, clearly, and briefly as humanly said as possible. "
    #     "Just provide the final caption directly and please responds only using lowercase letter without using any punctuation. "
    #     "Remember to remove any elements from the list that you think are duplicates, for example: if there are two list members that mention sneakers then ignore one of them."
    # )
    
    # <B> with punctuation, with "person is wearing" => deprecated i guess?
    message_content = (
        "I have a list of captions that describe each segmented part of a fully clothed person in an image. "
        "The list of captions is: " + caption_str + ". "
        "Your task is to combine these captions into one cohesive caption that describes "
        "the clothing worn by the person in the image as humanly said as possible. "
        "Just provide the final caption directly and please responds only using lowercase letter. "
        "Remember to remove any elements from the list that you think are duplicates, for example: if there are two list members that mention sneakers then ignore one of them."
    )
    
    # # <C> with punctuation, no "person is wearing" => ideal for questionnaire
    # message_content = (
    #     "I have a list of captions that describe each segmented part of a fully clothed person in an image. "
    #     "The list of captions is: " + caption_str + ". "
    #     "Your task is to combine these captions into one cohesive caption that describes "
    #     "the clothing worn by the person in the image concisely, clearly, and briefly as humanly said as possible. "
    #     "Just provide the final caption directly and please responds only using lowercase letter"
    #     "Remember to remove any elements from the list that you think are duplicates, for example: if there are two list members that mention sneakers then ignore one of them."
    #     "Please do not use phrase like 'the person is wearing..', instead please combine the provided captions only using necessary conjunctions"
    # )

    completion = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant which masters text rewriting"},
            {"role": "user", "content": message_content}
        ]
    )
    return completion.choices[0].message.content.strip() # aku tambahain strip() untuk menghapus extra spaces

In [40]:
import torch
from PIL import Image
import numpy as np
import cv2
import time

# === LOAD PREDICTOR, PROCESSOR, DAN MODEL BLIP-2 SEBELUM INI ===
# Misalnya:
# predictor = ... # Mask R-CNN atau Detectron2 predictor
# processor = ... # BLIP-2 processor (dari transformers)
# model = ...     # BLIP-2 model
# client = ...    # OpenAI client

# --- Awal pengukuran waktu ---
start_time = time.time()

# === INPUT GAMBARNYA ===
# image_path = "kemejadanchinosRGB.png" # gambar 1
# image_path = "inferenceblip2/435851fb-894a-4030-a981-afe9ad4c3502.jpeg" # gambar 2
# image_path = "inferencepemaduancaption/image_0672.jpg" # gambar 3
# image_path = "inferencepemaduancaption/0207933.jpg" # gambar 4
image_path = "inferencepemaduancaption/1091631.jpg" # gambar 5
image = cv2.imread(image_path)

# === 1. Segmentasikan gambar ===
crops_meta = segment_image_in_memory(image)

# === 2. Lakukan captioning pada tiap hasil crop ===
captions = batch_caption_inference(
    crops_meta=crops_meta,
    processor=processor,
    model=model,
    device=DEVICE,
    max_length=MAX_LENGTH,
    batch_size=CAPTION_BATCH_SIZE
)

# === 3. Gabungkan caption (opsional, jika ingin satu kalimat deskriptif) ===
combined_caption = combine_instance_captions(captions)

# --- Awal pengukuran waktu ---
# start_time = time.time()

# Jika ingin gabungkan dengan LLM (opsional)
final_caption = llm_combine(combined_caption)

# --- Akhir pengukuran waktu ---
end_time = time.time()
total_duration = end_time - start_time

# === OUTPUT ===
print("Captions per segment:")
for cap in captions:
    print("-", cap)

print("\nCombined (manual):", combined_caption)
print("\nCombined (LLM):", final_caption)
print(f"\nTotal time taken: {total_duration:.2f} seconds")

Captions per segment:
- black jeans with a zipper down the leg and a belt around the
- red brown sunglasses
- brown sweater with a white collar and collarless sweater with a white
- black white boston boston boston boston boston b
- white red viper's wing collar fang necklace
- black white canvas converse all star sneaker

Combined (manual): black jeans with a zipper down the leg and a belt around the, red brown sunglasses, brown sweater with a white collar and collarless sweater with a white, black white boston boston boston boston boston b, white red viper's wing collar fang necklace, black white canvas converse all star sneaker

Combined (LLM): the person is wearing black jeans with a zipper down the leg and a belt, a brown sweater featuring a white collar, red brown sunglasses, a white and red viper's wing collar fang necklace, and black and white canvas converse all star sneakers.

Total time taken: 20.96 seconds
